In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
import cv2
import pytesseract
import numpy as np
import medspacy

from PIL import Image
from medspacy.visualization import visualize_ent

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

#nlp = medspacy.load("en_info_3700_i2b2_2012")


In [4]:
import spacy
nlp2=spacy.load('en_core_web_sm')
nlp=medspacy.load(nlp2)
nlp2.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'medspacy_pyrush',
 'medspacy_target_matcher',
 'medspacy_context']

In [2]:
print(nlp.pipe_names)

NameError: name 'nlp' is not defined

In [41]:
def extract_text(image_path):
    img = cv2.imread(image_path)
    
    filename = "temp.png"
    cv2.imwrite(filename, img)
    text = pytesseract.image_to_string(Image.open(filename))
    return text

In [42]:
def extract_medication_info(text):
    doc = nlp2(text)
    medications = [ent.text for ent in doc.ents]
    #dosages = [dos.text for dos in doc._.medspacy_dosages]
    return medications

In [43]:
image_path = r"C:\Users\rrsh8\Downloads\test-ner\12.jpg"
text = extract_text(image_path)



error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [8]:
print(text)

Patient Name: John Smith
Address) 400 E 3rd Street,
Duluth, MN 56804

Rx Amoxicillin 250 mg tablets a
TT tablets p.o. Tid. x 7 days

Donot Ref! _X (Sig) mo.
Ref Times D.EA. Number
Date 10/3/00 \ Print Last Name Johns _




In [9]:
import spacy
from spacy import displacy
from collections import Counter
import en_ner_bc5cdr_md

# Load the small scispacy model
nlp = en_ner_bc5cdr_md.load()

# Define the text to process
text2 = """(1) Primary Hypothyroidism (2) Chronic autoimmune Thyroiditis (3)
Post Thyroideciomy (4) Post Radio-lodine treatment

(Subclinical Hypothyroidism (2) Patient with insufficient thyroid
hormone replacement therapy (3) In cases of Autoimmune/Hashimoto
thyroiditis (4). Isolated increase in TSH levels can be due to Subclinical
inflammation, drugs like amphetamines, Iodine containing drug and
dopamine antagonist e.g. domperidone and other physiological reasons.

(1) Secondary and Tertiary Hypothyroidism

(1) Primary Hyperthyroidism (Graves Disease) (2) Multinodular Goite
G)Toxie Nodular Goitre (4) Thyroiditis (5) Over treatment of thyroid
hormone (6) Drug effect e.¢ Glucocorticoids, dopamine, T4
replacement therapy (7) First trimester of Pregnancy

(1) Subclinical Hyperthyroidism

(I) TSH secreting pituitary adenoma (2) TRH secreting tumor

(1) Central Hypothyroidism (2) Euthyroid sick syndrome (3) Recent
treatment for Hyperthyroidism

(TS thyrotoxicosis (2) Non-Thyroidal illness

(1) 14 Ingestion (2) Thyroiditis (3) Interfesine Anal TPO amtibodies

"""

# Process the text with scispacy
doc = nlp(text)

# Identify medication entities in the text
medications = []
for ent in doc.ents:
    if ent.label_ == "CHEMICAL":
        medications.append(ent.text)

# Display the identified medications
if medications:
    medication_counts = Counter(medications)
    for medication, count in medication_counts.items():
        print(f"{medication}: {count}")
else:
    print("No medications found in the text.")

Radio-lodine: 1
amphetamines: 1
dopamine: 2
domperidone: 1
Interfesine: 1
TPO: 1


In [4]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [44]:
nlp3 = medspacy.load("en_core_med7_trf")

In [45]:
doc = nlp3("John Brown Main Street Libertyville Maryland Phone Name Mary Smith Date Jan Address Broad Street R Lipitor mg Tabs No Sig tab I day Refill times Generic available yes IM Brown DEA No State License No")



In [46]:


visualize_ent(doc)



In [6]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp_custom = spacy.blank("en") # load a new spacy model
db = DocBin()

In [7]:
import json
f = open('annotations.json')
TRAIN_DATA = json.load(f)

In [9]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp_custom.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk('training_data.spacy')

100%|██████████| 100/100 [00:00<00:00, 5274.46it/s]


In [10]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     87.10   41.67   36.76   48.08    0.42
 87     200          5.88   1099.69  100.00  100.00  100.00    1.00
187     400          0.00      0.00  100.00  100.00  100.00    1.00
287     600          0.00      0.00  100.00  100.00  100.00    1.00
451     800          0.00      0.00  100.00  100.00  100.00    1.00
651    1000          0.00      0.00  100.00  100.00  100.00    1.00
851    1200          0.00      0.00  100.00  100.00  100.00    1.00
1051    1400          0.00      0.00  100.00  100.00  100.00    1.00
1251    1600          0.00      0.0

[2023-05-27 10:33:19,460] [INFO] Set up nlp object from config
[2023-05-27 10:33:19,467] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-05-27 10:33:19,476] [INFO] Created vocabulary
[2023-05-27 10:33:19,487] [INFO] Finished initializing nlp object
[2023-05-27 10:33:19,679] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [5]:
nlp_ner_custom = spacy.load("model-best")

In [ ]:
for token in doc:
    print(token.text, token.ent_iob_)

In [20]:
image_path = r"C:\Users\rrsh8\Downloads\test-ner\Sample-Electronic-Prescription.png"
text4 = extract_text(image_path)

In [9]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re

def preprocess_text(text):
    # Load SpaCy pipeline
    nlp = spacy.load("en_core_web_sm")

    # Tokenization, lowercasing, and lemmatization
    doc = nlp(text)
    tokens = [token.lemma_.strip() for token in doc]
    # Remove stop words and non-alphabetic tokens
    tokens = [token for token in tokens if token.isalpha() and token not in STOP_WORDS]

    # Remove punctuation and special characters
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]

    # Remove extra whitespace
    tokens = [token.strip() for token in tokens if token.strip()]

    # Join tokens back into a cleaned text
    cleaned_text = ' '.join(tokens)

    return cleaned_text


In [10]:
prepro=preprocess_text(text)
print(str(prepro))

patient Name John Smith Address e Street Duluth MN Rx Amoxicillin mg tablet TT tablet Tid x day Donot Ref X Sig mo Ref Times number Date Print Name Johns


In [12]:
doc3=nlp(str(prepro))

ValueError: [E043] Refusing to write to token.sent_start if its document is parsed, because this may cause inconsistent state.

In [72]:
from spacy import displacy
displacy.render(doc3, style="ent", jupyter=True)

[JKUAT, HOSPITAL, 
, P.O, BOX, 62000, NAIROBI, KENYA, Tel, 067, 52181, 4, FAX, 52164, THIKA, 
, Patient, Prescription, 

, Date, 10/09/2006, No:604, WAIRIA, STEPHEN, 

, Drug, Number, Units, Number, Times, Number, Days, 
, Chlorzoxazone, 10.0qm, 2, Twice, 3, 
, Diplafenac, 12.0gm, 2, 6, 

, Doctor, Signature, 

]


In [1]:
import spacy
from spacy_streamlit import visualize_tokens

nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a text")
visualize_tokens(doc, attrs=["text", "pos_", "dep_", "ent_type_"])

2023-06-09 21:38:46.101 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
c:\Users\rrsh8\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\rrsh8\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-09 21:38:47.376 
  command:

    streamlit run C:\Users\rrsh8\AppData\Roaming\Python\Python311\site-packages\ipykernel_l

In [18]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rrsh8\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [19]:
from spacy.lang.en.stop_words import STOP_WORDS

In [34]:
def clean_text(text):
    # Load the spaCy English model
    nlp = spacy.load('en_core_web_sm')

    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove commas and other unwanted characters
    text = re.sub(r'[,;:]+', '', text)

    # Tokenize the text using spaCy
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if not token.is_stop]
    text = ' '.join(filtered_tokens)

    return text

In [39]:
cleaned_text = clean_text(text)
print(str(cleaned_text))

Patient John Smith Address ) 400 E 3rd Street Duluth MN 56804 Rx Amoxicillin 250 mg tablets TT tablets p.o . Tid . x 7 days Donot Ref ! _ X ( Sig ) mo . Ref Times D.EA . Number Date 10/3/00 \ Print Johns _


In [38]:
doc3=nlp(text)

ValueError: [E043] Refusing to write to token.sent_start if its document is parsed, because this may cause inconsistent state.